In [ ]:
import sys
from pathlib import Path

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

IN_DIR = DIR/'examples'/'inputs'
OUT_DIR = DIR/'examples'/'outputs'

%load_ext autoreload
%autoreload 2

high_definition = False


In [ ]:
# Download elevation data

bounds = [174, -37, 176, -36]
names = wt.get_srtm_tile_names(bounds)
print('Tiles to download =', names)

username = None
password = None
if high_definition:
    path = IN_DIR/'elevations-hd'
    try:
        username = wt.get_secret('NASA_USERNAME')
        password = wt.get_secret('NASA_PASSWORD')
    except:
        pass
else:
    path = IN_DIR/'elevations-sd'
    
wt.download_elevation_data_nasa(bounds, path, high_definition=high_definition,
  username=username, password=password)

In [ ]:
# Create SPLAT transmitters data

in_path = IN_DIR/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
wt.create_splat_transmitter_data(in_path, out_path)
% ls {out_path}

In [ ]:
# Create SPLAT elevation data

if high_definition:
    in_path = IN_DIR/'elevations-hd'
else:
    in_path = IN_DIR/'elevations-sd'
out_path = OUT_DIR/'splat_files'

wt.create_splat_elevation_data(in_path, out_path, high_definition=high_definition)
% ls {out_path}

In [2]:
# Create coverage reports

transmitter_names = ['MyNetwork_11']
in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR/'coverage_reports'
wt.create_coverage_reports(in_path, out_path, transmitter_names, 
  high_definition=high_definition)
% ls {out_path}

Timing create_coverage_reports...
2016-08-09 12:45:08.545321   Began process
2016-08-09 12:48:08.527151   Finished in 2.98 min
MyNetwork_11-ck.ppm           MyNetwork_11.kml
MyNetwork_11-site_report.txt  MyNetwork_11.ppm
MyNetwork_11.geo


In [ ]:
# Post-process coverage reports

path = OUT_DIR/'coverage_reports'
wt.postprocess_coverage_reports(path, save_ppm=True)
% ls {path}

In [20]:
import re
import subprocess


# Get PPM image lon-lat bounds from KML
path = OUT_DIR/'coverage_reports'/'MyNetwork_11.kml'
with path.open() as src:
    kml = src.read()

north = re.search(r"<north>([0-9-][0-9\.]*)<\/north>", kml).group(1)
south = re.search(r"<south>([0-9-][0-9\.]*)<\/south>", kml).group(1)
east = re.search(r"<east>([0-9-][0-9\.]*)<\/east>", kml).group(1)
west = re.search(r"<west>([0-9-][0-9\.]*)<\/west>", kml).group(1)
print(north, south, east, west)

# Convert PPM (with lon-lat bounds) to GeoTiff via GDAL
ppm = 'MyNetwork_11.ppm'
tif = ppm.replace('.ppm', '.tif')
epsg = '4326'
args = ['gdal_translate', '-of', 'Gtiff', '-a_ullr', west, north, east, south, 
 '-a_srs', epsg, ppm, tif]
print(" ".join(args))
subprocess.run(args, cwd=str(path.parent),
  stdout=subprocess.PIPE, universal_newlines=True, check=True)

# See http://www.gdal.org/gdal_translate.html and https://gis.stackexchange.com/questions/118180/convert-png-to-geotiff-using-gdal
 
#gdal_translate -of Gtiff -a_ullr LEFT_LON UPPER_LAT RIGHT_LON LOWER_LAT -a_srs EPSG_PROJ INPUT_PNG_FILE OUTPUT_GTIFF_FILE. 

-35.00083 -38.00000 177.00000 173.00000
gdal_translate -of Gtiff -a_ullr 173.00000 -35.00083 177.00000 -38.00000 -a_srs 4326 MyNetwork_11.ppm MyNetwork_11.tif


CalledProcessError: Command '['gdal_translate', '-of', 'Gtiff', '-a_ullr', '173.00000', '-35.00083', '177.00000', '-38.00000', '-a_srs', '4326', 'MyNetwork_11.ppm', 'MyNetwork_11.tif']' returned non-zero exit status 1